In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

# Загрузка данных
data = pd.read_csv("../data/processed/train.csv")
data

,total_meters,price
0,40.1,35500000
1,79.0,75000000
2,21.6,4700000
3,51.7,15000000
4,43.0,30800000
...,...,...
545,30.0,12900000
546,31.9,17490000
547,39.6,10900000
548,32.5,12930000


In [8]:
# Стандартизация признака
scaler = StandardScaler()
data['total_meters'] = scaler.fit_transform(data[['total_meters']])

# Разделение на признаки и целевую переменную
X = data[['total_meters']]  # только один признак - площадь
y = data['price']

# Убираем крайние выбросы
q_low, q_high = y.quantile([0.01, 0.99])
mask = (y >= q_low) & (y <= q_high)
X, y = X[mask], y[mask]

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Лог-трансформация целевой
y_train_log = np.log1p(y_train)

# Создание и обучение модели

# Задаём сетку альфа для RidgeCV
alphas = np.logspace(-3, 3, 13)

# Собираем pipeline: стандартизация + подбор α
model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge_cv", RidgeCV(alphas=alphas, cv=5, scoring="r2"))
])

# Обучаем и выбираем лучшее α
model.fit(X_train, y_train_log)
best_alpha = model.named_steps["ridge_cv"].alpha_
print(f"Лучшее α для Ridge: {best_alpha:.4f}")

cv_scores = cross_val_score(model, X_train, y_train_log, cv=5, scoring="r2")
print(f"CV R² on TRAIN: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Предсказание на тестовой выборке
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # обратное exp

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
# todo: use logging
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

ridge_model = model.named_steps["ridge_cv"]

# Коэффициенты модели
print(f"Коэффициент при площади: {ridge_model.coef_[0]:.2f}")
print(f"Свободный член: {ridge_model.intercept_:.2f}")

Лучшее α для Ridge: 10.0000
CV R² on TRAIN: 0.474 ± 0.094
Среднеквадратичная ошибка (MSE): 65321064007813.69
Корень из среднеквадратичной ошибки (RMSE): 8082144.77
Коэффициент детерминации R²: 0.574020
Средняя ошибка предсказания: 5681474.46 рублей
Коэффициент при площади: 0.40
Свободный член: 16.50


In [9]:
import joblib
# Сохранение модели
model_path = '../models/linear_regression_v4.pkl'

joblib.dump(model, model_path)
print("Модель сохранена в файл f{model_path}")

# Загрузка модели
loaded_model = joblib.load(model_path)
print("Модель загружена из файла")

Модель сохранена в файл f{model_path}
Модель загружена из файла
